In [ ]:
# ===== 모듈 3: 협업 분석 모듈 구현 =====
# 주피터 노트북 환경에서 실행

# 필요한 라이브러리 import
from typing import Annotated, List, Literal, TypedDict, Dict, Optional
from langchain_core.messages import HumanMessage 
import operator
from langgraph.graph import StateGraph, START, END

# 실제 구현시 아래 주석 해제
# from db_utils import *
# from llm_utils import *

print("모듈 3: 협업 분석 모듈 구현 시작")


# --- Module3AgentState 정의 ---
class Module3AgentState(TypedDict):
    """
    모듈 3 (협업 분석 모듈)의 내부 상태를 정의합니다.
    """
    messages: Annotated[List[HumanMessage], operator.add] 

    report_type: Literal["quarterly", "annual"] 
    team_id: int 
    period_id: int 
    
    # 입력 데이터 ID들
    target_task_summary_ids: List[int] 
    target_team_kpi_ids: List[int] 

    # 협업 관계 데이터
    collaboration_relationships: List[Dict] = []
    
    # 개인별 협업 분석 결과
    individual_collaboration_analysis: Dict[str, Dict] = {}
    
    # 팀 협업 네트워크 결과
    team_collaboration_network: List[Dict] = []
    
    # 레포트 저장 관련 ID들
    feedback_report_id: int = None 
    team_evaluation_id: int = None 
    final_evaluation_report_id: int = None 
    updated_temp_evaluation_ids_list: List[int] = [] 

print("State 정의 완료")



# ===== 서브모듈 1: 데이터 수집 서브모듈 =====
def collaboration_data_collection_submodule(state: Module3AgentState) -> Module3AgentState:
    """
    협업 분석을 위한 기초 데이터를 수집합니다.
    - 동일 KPI 내 Task 그룹핑으로 잠재적 협업 관계 파악
    """
    print("=== 서브모듈 1: 협업 데이터 수집 시작 ===")
    
    target_team_kpi_ids = state["target_team_kpi_ids"]
    period_id = state["period_id"]
    
    collaboration_relationships = []
    
    # 각 KPI별로 협업 관계 1차 구성
    for team_kpi_id in target_team_kpi_ids:
        print(f"KPI {team_kpi_id} 내 Task 협업 관계 분석 중...")
        
        # 동일 KPI 내 모든 Task 조회 (모듈 2에서 사용한 함수 재활용)
        kpi_tasks = fetch_tasks_for_kpi(team_kpi_id, period_id)
        
        if len(kpi_tasks) > 1:  # 2개 이상 Task가 있어야 협업 가능
            # KPI 내 모든 Task를 잠재적 협업 관계로 설정
            for task in kpi_tasks:
                # 동일 KPI 내 다른 팀원들을 잠재적 협업자로 설정
                potential_collaborators = [
                    {"emp_no": t["emp_no"], "emp_name": t["emp_name"]} 
                    for t in kpi_tasks if t["emp_no"] != task["emp_no"]
                ]
                
                collaboration_relationships.append({
                    "task_id": task["task_id"],
                    "task_summary_id": task.get("task_summary_Id"),
                    "emp_no": task["emp_no"],
                    "emp_name": task["emp_name"],
                    "task_name": task["task_name"],
                    "task_summary": task.get("task_summary", ""),
                    "team_kpi_id": team_kpi_id,
                    "potential_collaborators": potential_collaborators,
                    "collaboration_confirmed": False  # LLM 분석 전
                })
        else:
            print(f"KPI {team_kpi_id}는 Task 1개뿐이라 협업 관계 없음")
    
    print(f"총 {len(collaboration_relationships)}개 잠재적 협업 관계 수집 완료")
    
    messages = state.get("messages", []) + [HumanMessage(content="모듈 3: 협업 데이터 수집 완료")]
    return {
        "messages": messages, 
        "collaboration_relationships": collaboration_relationships
    }


# ===== 서브모듈 2: 개인 협업 분석 서브모듈 =====
def individual_collaboration_analysis_submodule(state: Module3AgentState) -> Module3AgentState:
    """
    Task Summary를 LLM으로 분석하여 실제 협업 관계를 확인하고,
    개인별 협업 패턴을 분석합니다.
    """
    print("=== 서브모듈 2: 개인 협업 분석 시작 ===")
    
    collaboration_relationships = state["collaboration_relationships"]
    team_id = state["team_id"]
    
    # 1. LLM으로 실제 협업 관계 확인
    confirmed_collaborations = []
    
    for relation in collaboration_relationships:
        task_summary = relation["task_summary"]
        potential_collaborators = relation["potential_collaborators"]
        
        if task_summary and potential_collaborators:
            print(f"Task '{relation['task_name']}' 협업 관계 LLM 분석 중...")
            
            # LLM 호출하여 실제 협업 여부 및 협업자 확인
            llm_collaboration_result = call_llm_for_collaboration_detection(
                task_summary=task_summary,
                task_name=relation["task_name"],
                potential_collaborators=[c["emp_name"] for c in potential_collaborators],
                emp_name=relation["emp_name"],
                emp_no=relation["emp_no"]
            )
            
            if llm_collaboration_result.get("is_collaboration", False):
                confirmed_collaborations.append({
                    **relation,
                    "collaboration_confirmed": True,
                    "actual_collaborators": llm_collaboration_result.get("collaborators", []),
                    "collaboration_content": llm_collaboration_result.get("collaboration_content", ""),
                    "collaboration_intensity": llm_collaboration_result.get("intensity", "보통")
                })
                print(f"✓ 협업 확인: {relation['emp_name']} ↔ {llm_collaboration_result.get('collaborators', [])}")
            else:
                print(f"✗ 협업 없음: {relation['task_name']}")
    
    print(f"총 {len(confirmed_collaborations)}개 실제 협업 관계 확인 완료")
    
    # 2. 개인별 협업 통계 분석
    individual_analysis = {}
    all_team_members = fetch_employees_by_team_id(team_id)
    
    for member in all_team_members:
        emp_no = member["emp_no"]
        emp_name = member["emp_name"]
        
        # 해당 개인의 모든 Task 수 (협업 + 단독)
        person_all_tasks = [r for r in collaboration_relationships if r["emp_no"] == emp_no]
        total_tasks = len(person_all_tasks)
        
        # 해당 개인이 참여한 협업 Task 수
        person_collaboration_tasks = [c for c in confirmed_collaborations if c["emp_no"] == emp_no]
        collaboration_tasks = len(person_collaboration_tasks)
        
        # 공동 Task 수행률 계산
        collaboration_rate = round((collaboration_tasks / total_tasks * 100), 2) if total_tasks > 0 else 0.0
        
        # 핵심 협업자 분석 (가장 많이 협업한 동료 찾기)
        collaborator_count = {}
        for collab_task in person_collaboration_tasks:
            for collaborator in collab_task.get("actual_collaborators", []):
                collaborator_count[collaborator] = collaborator_count.get(collaborator, 0) + 1
        
        # 상위 2-3명 핵심 협업자
        sorted_collaborators = sorted(collaborator_count.items(), key=lambda x: x[1], reverse=True)
        key_collaborators = [name for name, count in sorted_collaborators[:3]]
        
        # 협업 편중도 계산 (상위 1명과의 협업 비율)
        if sorted_collaborators and collaboration_tasks > 0:
            top_collaborator_ratio = sorted_collaborators[0][1] / collaboration_tasks
            if top_collaborator_ratio >= 0.7:
                bias_level = "높음"
            elif top_collaborator_ratio >= 0.4:
                bias_level = "보통"
            else:
                bias_level = "낮음"
        else:
            bias_level = "해당없음"
        
        individual_analysis[emp_no] = {
            "emp_name": emp_name,
            "total_tasks": total_tasks,
            "collaboration_tasks": collaboration_tasks,
            "collaboration_rate": collaboration_rate,
            "key_collaborators": key_collaborators,
            "bias_level": bias_level,
            "collaborator_count": collaborator_count
        }
        
        print(f"{emp_name}: 총 {total_tasks}개 Task, 협업률 {collaboration_rate}%, 편중도 {bias_level}")
    
    messages = state.get("messages", []) + [HumanMessage(content="모듈 3: 개인 협업 분석 완료")]
    return {
        "messages": messages,
        "collaboration_relationships": confirmed_collaborations,  # 확정된 협업 관계로 업데이트
        "individual_collaboration_analysis": individual_analysis
    }


# ===== 서브모듈 3: 팀 협업 네트워크 서브모듈 =====
def team_collaboration_network_submodule(state: Module3AgentState) -> Module3AgentState:
    """
    팀 전체의 협업 네트워크를 분석하고 팀장용 협업 네트워크 표를 생성합니다.
    """
    print("=== 서브모듈 3: 팀 협업 네트워크 분석 시작 ===")
    
    individual_analysis = state["individual_collaboration_analysis"]
    confirmed_collaborations = state["collaboration_relationships"]
    
    # 팀 협업 네트워크 표 생성
    team_network = []
    
    for emp_no, analysis in individual_analysis.items():
        emp_name = analysis["emp_name"]
        
        # LLM 호출하여 종합 평가 코멘트 생성
        collaboration_comment = call_llm_for_individual_collaboration_comment(
            emp_name=emp_name,
            emp_no=emp_no,
            total_tasks=analysis["total_tasks"],
            collaboration_rate=analysis["collaboration_rate"],
            key_collaborators=analysis["key_collaborators"],
            bias_level=analysis["bias_level"],
            collaboration_details=[c for c in confirmed_collaborations if c["emp_no"] == emp_no]
        )
        
        team_network.append({
            "emp_no": emp_no,
            "emp_name": emp_name,
            "total_tasks": analysis["total_tasks"],
            "collaboration_rate": analysis["collaboration_rate"],
            "key_collaborators": ", ".join(analysis["key_collaborators"]) if analysis["key_collaborators"] else "없음",
            "bias_level": analysis["bias_level"],
            "collaboration_comment": collaboration_comment.get("comment", f"{emp_name}님의 협업 분석 생성 실패")
        })
        
        print(f"{emp_name} 협업 네트워크 분석 완료")
    
    messages = state.get("messages", []) + [HumanMessage(content="모듈 3: 팀 협업 네트워크 분석 완료")]
    return {
        "messages": messages,
        "team_collaboration_network": team_network
    }

# ===== 서브모듈 4: 협업 기여도 종합 분석 서브모듈 =====
def collaboration_comprehensive_analysis_submodule(state: Module3AgentState) -> Module3AgentState:
    """
    최종 전 중간 평가용 협업 기여도 종합 분석을 수행합니다.
    """
    print("=== 서브모듈 4: 협업 기여도 종합 분석 시작 ===")
    
    report_type = state["report_type"]
    individual_analysis = state["individual_collaboration_analysis"]
    confirmed_collaborations = state["collaboration_relationships"]
    
    if report_type == "annual":
        # 연말 보고서인 경우만 최종 전 중간 평가 데이터 생성
        comprehensive_analysis = {}
        
        for emp_no, analysis in individual_analysis.items():
            emp_name = analysis["emp_name"]
            
            # Peer Talk 키워드는 모듈 4에서 처리예정이므로 일단 placeholder
            peer_talk_keywords = "모듈4에서_처리예정"
            
            # 협업 종합 평가 LLM 생성
            comprehensive_comment = call_llm_for_comprehensive_collaboration_evaluation(
                emp_name=emp_name,
                emp_no=emp_no,
                annual_collaboration_data=analysis,
                collaboration_tasks_details=[c for c in confirmed_collaborations if c["emp_no"] == emp_no]
            )
            
            comprehensive_analysis[emp_no] = {
                "emp_name": emp_name,
                "total_tasks": analysis["total_tasks"],
                "collaboration_rate": analysis["collaboration_rate"],
                "collaborator_count": len(analysis["key_collaborators"]),
                "key_collaborators": ", ".join(analysis["key_collaborators"]) if analysis["key_collaborators"] else "없음",
                "peer_talk_keywords": peer_talk_keywords,  # 모듈 4에서 업데이트 예정
                "bias_level": analysis["bias_level"],
                "comprehensive_comment": comprehensive_comment.get("comment", f"{emp_name}님의 종합 협업 평가 생성 실패")
            }
            
            print(f"{emp_name} 협업 종합 분석 완료")
        
        # state에 종합 분석 결과 추가
        state["comprehensive_collaboration_analysis"] = comprehensive_analysis
    else:
        print("분기별 보고서이므로 협업 종합 분석 건너뜀")
    
    messages = state.get("messages", []) + [HumanMessage(content="모듈 3: 협업 기여도 종합 분석 완료")]
    return {"messages": messages}

# ===== 서브모듈 5: 포맷터 서브모듈 =====
def collaboration_formatter_submodule(state: Module3AgentState) -> Module3AgentState:
    """
    협업 분석 결과를 최종 포맷으로 정리합니다.
    """
    print("=== 서브모듈 5: 협업 분석 포맷팅 시작 ===")
    
    # 결과 출력 (디버깅용)
    team_network = state.get("team_collaboration_network", [])
    print("\n=== 팀 협업 네트워크 결과 ===")
    for member in team_network:
        print(f"• {member['emp_name']}: 총 {member['total_tasks']}개 Task, 협업률 {member['collaboration_rate']}%, 편중도 {member['bias_level']}")
    
    messages = state.get("messages", []) + [HumanMessage(content="모듈 3: 포맷팅 완료")]
    return {"messages": messages}

print("모든 서브모듈 함수 정의 완료")


# ===== LLM 호출 함수들 (llm_utils.py에 추가될 함수들) =====

def call_llm_for_collaboration_detection(task_summary: str, task_name: str, potential_collaborators: List[str], emp_name: str, emp_no: str) -> Dict:
    """
    Task Summary를 분석하여 실제 협업 여부와 협업자를 확인합니다.
    """
    print(f"LLM Call (Collaboration Detection): '{task_name}' 협업 분석")
    
    # 실제 구현시 LLM 호출 로직
    # 임시 mock 응답
    mock_response = {
        "is_collaboration": True,
        "collaborators": potential_collaborators[:2] if len(potential_collaborators) >= 2 else potential_collaborators,
        "collaboration_content": f"{emp_name}님이 {', '.join(potential_collaborators[:2])}님과 협업하여 {task_name} 수행",
        "intensity": "보통"
    }
    
    return mock_response


def call_llm_for_individual_collaboration_comment(emp_name: str, emp_no: str, total_tasks: int, collaboration_rate: float, key_collaborators: List[str], bias_level: str, collaboration_details: List[Dict]) -> Dict:
    """
    개인의 협업 패턴에 대한 종합 코멘트를 생성합니다.
    """
    print(f"LLM Call (Individual Collaboration Comment): '{emp_name}' 협업 코멘트 생성")
    
    # 실제 구현시 LLM 호출 로직
    # 임시 mock 응답
    mock_comment = f"{emp_name}({emp_no})님은 총 {total_tasks}개 Task 중 {collaboration_rate}%의 협업률을 보이며, 주로 {', '.join(key_collaborators)}님과 협업합니다. 협업 편중도는 {bias_level} 수준입니다."
    
    return {"comment": mock_comment}


def call_llm_for_comprehensive_collaboration_evaluation(emp_name: str, emp_no: str, annual_collaboration_data: Dict, collaboration_tasks_details: List[Dict]) -> Dict:
    """
    연간 협업 데이터를 종합하여 최종 평가 코멘트를 생성합니다.
    """
    print(f"LLM Call (Comprehensive Collaboration): '{emp_name}' 연간 협업 종합 평가")
    
    # 실제 구현시 LLM 호출 로직
    # 임시 mock 응답
    mock_comment = f"{emp_name}({emp_no})님의 연간 협업 성과는 우수하며, 다양한 동료들과 효과적으로 협업을 수행했습니다."
    
    return {"comment": mock_comment}

print("LLM 호출 함수들 정의 완료")


# ===== 워크플로우 생성 =====
def create_module3_graph():
    """모듈 3 그래프 생성 및 반환"""
    print("모듈 3 워크플로우 그래프 생성 중...")
    
    module3_workflow = StateGraph(Module3AgentState)
    
    # 노드 추가 (노드명 통일)
    module3_workflow.add_node("data_collection", collaboration_data_collection_submodule)
    module3_workflow.add_node("individual_analysis", individual_collaboration_analysis_submodule)
    module3_workflow.add_node("team_network", team_collaboration_network_submodule)
    module3_workflow.add_node("comprehensive_analysis", collaboration_comprehensive_analysis_submodule)
    module3_workflow.add_node("formatter", collaboration_formatter_submodule)
    
    # 엣지 정의
    module3_workflow.add_edge(START, "data_collection")
    module3_workflow.add_edge("data_collection", "individual_analysis")
    module3_workflow.add_edge("individual_analysis", "team_network")
    module3_workflow.add_edge("team_network", "comprehensive_analysis")
    module3_workflow.add_edge("comprehensive_analysis", "formatter")
    module3_workflow.add_edge("collaboration_formatter", END)
    
    print("모듈 3 워크플로우 그래프 생성 완료")
    return module3_workflow.compile()


# ===== 테스트 실행 함수 =====
def test_module3_quarterly():
    """모듈 3 분기별 테스트 실행"""
    print("\n" + "="*50)
    print("모듈 3 분기별 테스트 실행 시작")
    print("="*50)
    
    # State 정의
    test_state = Module3AgentState(
        messages=[HumanMessage(content="모듈 3 협업 분석 테스트 시작")],
        report_type="quarterly",
        team_id=1,
        period_id=2,
        target_task_summary_ids=[1, 5, 9, 13, 17, 21, 25, 29, 2, 6, 10, 14, 18, 22, 26, 30],
        target_team_kpi_ids=[1, 2, 3],
        team_evaluation_id=101
    )

    # 그래프 생성 및 실행
    try:
        module3_graph = create_module3_graph()
        print("모듈 3 실행 시작...")
        result = module3_graph.invoke(test_state)
        print("모듈 3 실행 완료!")
        print(f"최종 메시지: {result['messages'][-1].content}")
        return result
    except Exception as e:
        print(f"모듈 3 실행 중 오류 발생: {e}")
        return None

# 실행 (주피터 노트북에서)
if __name__ == "__main__":
    print("모듈 3 테스트 실행 준비 완료")
    print("실행하려면: test_module3_quarterly() 함수를 호출하세요")
    
    # 자동 실행 (주석 해제시)
    result = test_module3_quarterly()

모듈 3: 협업 분석 모듈 구현 시작
State 정의 완료
모든 서브모듈 함수 정의 완료
LLM 호출 함수들 정의 완료
모듈 3 테스트 실행 준비 완료
실행하려면: test_module3_quarterly() 함수를 호출하세요

모듈 3 분기별 테스트 실행 시작
모듈 3 워크플로우 그래프 생성 중...
모듈 3 워크플로우 그래프 생성 완료
모듈 3 실행 중 오류 발생: Found edge starting at unknown node 'collaboration_formatter'
